# Language-Emulating Autoencoder (LAE)

## Imports & Setups

In [3]:
import tensorflow as tf
import tensorflow.keras as keras

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from tqdm.notebook import tqdm

Optional: instantiate TPU instances for accelerated training.

In [4]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

## Constants

In [6]:
IMAGE_DIM = 32   # width and height dimension of the input image, must be a power of 2
NUM_CHANNELS = 3  # number of channels in the input image, usually 1 or 3
VOCAB_SIZE = 128  # number of token slots for the encoded language, must be a power of 2
MAX_SEQ_LEN = 64  # maximum sequence length for the encoded language, must be a power of 2

## Load Dataset

CIFAR-10/100 or MNIST from Keras:

In [8]:
(X_train, y_train), (X_valid, y_valid) = keras.datasets.cifar100.load_data(label_mode="fine") #keras.datasets.mnist.load_data()

if not (X_train.shape[1] == X_train.shape[2] == IMAGE_DIM):

    new_X_train = np.zeros((len(X_train), IMAGE_DIM, IMAGE_DIM, NUM_CHANNELS))
    for i in tqdm(range(len(X_train))):
        new_X_train[i, :, :, 0:NUM_CHANNELS] = cv2.resize(X_train[i], (IMAGE_DIM, IMAGE_DIM))
    X_train = new_X_train

    new_X_valid = np.zeros((len(X_test), IMAGE_DIM, IMAGE_DIM, NUM_CHANNELS))
    for i in tqdm(range(len(X_test))):
        new_X_valid[i, :, :, 0:NUM_CHANNELS] = cv2.resize(X_test[i], (IMAGE_DIM, IMAGE_DIM))
    X_valid = new_X_valid
    
X_train = X_train / 255
X_valid = X_valid / 255

CalTech 101

In [4]:
# caltech = pd.read_csv('../input/caltech-101-csv/caltech101cleanedcsv.csv')

In [13]:
# x = np.array(caltech.drop('label', axis=1)).reshape((len(caltech), 200, 200, 1))

# train_indices = np.random.choice(np.arange(len(x)), size=round(0.8*len(x)), replace=False)
# valid_indices = [i for i in np.arange(len(x)) if (i not in train_indices)]
# X_train = x[train_indices]
# X_valid = x[valid_indices]

## Create Model Architecture

In [9]:
from tensorflow.keras import layers as L
from tensorflow.keras.models import Sequential, Model

In [56]:
# with tpu_strategy.scope():

# --------------------------------------------------------------
# CONSTRUCT CELLS ----------------------------------------------
# --------------------------------------------------------------

def halfReductionCell(inputShape, kernels, reduceAxis, identifier):
    kernels = int(kernels)
    inputShape = (int(inputShape[0]), int(inputShape[1]), int(inputShape[2]))
    inp = L.Input(inputShape)
    conv1 = L.Conv2D(kernels, (3, 3), activation='relu', padding='same')(inp)
    conv2 = L.Conv2D(kernels, (3, 3), activation='relu', padding='same')(conv1)
    if reduceAxis == 0:
        pool = L.MaxPooling2D((2, 1))(conv2)
    else:
        pool = L.MaxPooling2D((1, 2))(conv2)
    return Model(inputs=inp, outputs=pool, name=f'reduction_{identifier}')

def halfExpansionCell(inputShape, kernels, expandAxis, identifier):
    kernels = int(kernels)
    inputShape = (int(inputShape[0]), int(inputShape[1]), int(inputShape[2]))
    inp = L.Input(inputShape)
    conv1 = L.Conv2DTranspose(kernels, (3, 3), activation='relu', padding='same')(inp)
    conv2 = L.Conv2DTranspose(kernels, (3, 3), activation='relu', padding='same')(conv1)
    if expandAxis == 0:
        pool = L.UpSampling2D((2, 1))(conv2)
    else:
        pool = L.UpSampling2D((1, 2))(conv2)
    return Model(inputs=inp, outputs=pool, name=f'expansion_{identifier}')

def staticCell(inputShape, kernels, identifier):
    kernels = int(kernels)
    inputShape = (int(inputShape[0]), int(inputShape[1]), int(inputShape[2]))
    inp = L.Input(inputShape)
    conv1 = L.Conv2D(kernels, (3, 3), activation='relu', padding='same')(inp)
    conv2 = L.Conv2D(kernels, (3, 3), activation='relu', padding='same')(conv1)
    return Model(inputs=inp, outputs=conv2, name=f'static_{identifier}')

# --------------------------------------------------------------
# PRECOMPUTATIONS ----------------------------------------------
# --------------------------------------------------------------

maxIterations = int(max(np.log2(MAX_SEQ_LEN/IMAGE_DIM), np.log2(IMAGE_DIM)))


# --------------------------------------------------------------
# CONSTRUCT CUSTOM SYMBOL SELECTION LAYER ----------------------
# --------------------------------------------------------------

class SymbolSelection(tf.keras.layers.Layer):
    def __init__(self):
        super(SymbolSelection, self).__init__()

    def build(self, input_shape):
        pass
    
    @tf.custom_gradient
    def call(self, inputs):
        def grad(dy):
            return dy
        shape = (inputs.shape[1], inputs.shape[2])
        return tf.where(tf.equal(tf.reduce_max(inputs, axis=1, keepdims=True), inputs),
                        tf.constant(1., shape=shape),
                        tf.constant(0., shape=shape)), grad

# --------------------------------------------------------------
# CONSTRUCT ENCODER --------------------------------------------
# --------------------------------------------------------------

encInp = L.Input((IMAGE_DIM, IMAGE_DIM, NUM_CHANNELS))

# construct first branch
xa = L.Conv2D(8, (1, 1), padding='same', activation='relu')(encInp)
currFeatMap0Dim = IMAGE_DIM
currFeatMap1Dim = IMAGE_DIM
kernelSize = [8, 8] # [curr, next]
for iteration in range(maxIterations):
    if (kernelSize[0] < VOCAB_SIZE):
        kernelSize[1] = kernelSize[0] * 2
    if (currFeatMap0Dim > 1): # perform a reduction in the 0 axis
        xa = halfReductionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 0, f'enc_a{iteration}')(xa)
        currFeatMap0Dim /= 2
        kernelSize[0] = kernelSize[1] # perform update
    if (currFeatMap1Dim < MAX_SEQ_LEN): # perform an expansion in the 1 axis
        xa = halfExpansionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 1, f'enc_a{iteration}')(xa)
        currFeatMap1Dim *= 2
        kernelSize[0] = kernelSize[1] # perform update
xa = L.Reshape((MAX_SEQ_LEN, VOCAB_SIZE))(xa)

# construct second branch
xb = L.Conv2D(8, (1, 1), padding='same', activation='relu')(encInp)
currFeatMap0Dim = IMAGE_DIM
currFeatMap1Dim = IMAGE_DIM
kernelSize = [8, 8] # [curr, next]
for iteration in range(int(max(np.log2(MAX_SEQ_LEN/IMAGE_DIM), np.log2(IMAGE_DIM)))):
    if (kernelSize[0] < VOCAB_SIZE):
        kernelSize[1] = kernelSize[0] * 2
    if (currFeatMap1Dim > 1): # perform a reduction in the 0 axis
        xb = halfReductionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 1, f'enc_b{iteration}')(xb)
        currFeatMap1Dim /= 2
        kernelSize[0] = kernelSize[1] # perform update
    if (currFeatMap0Dim < MAX_SEQ_LEN): # perform an expansion in the 1 axis
        xb = halfExpansionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 0, f'enc_b{iteration}')(xb)
        currFeatMap0Dim *= 2
        kernelSize[0] = kernelSize[1] # perform update
xb = L.Reshape((MAX_SEQ_LEN, VOCAB_SIZE))(xb)

join = L.Add()([xa, xb])
lstm1 = L.LSTM(VOCAB_SIZE, return_sequences=True)(join)
lstm2 = L.LSTM(VOCAB_SIZE, return_sequences=True, activation='softmax')(lstm1)
out = SymbolSelection()(lstm2)

encoder = Model(inputs=encInp, outputs=out, name='encoder')


# --------------------------------------------------------------
# CONSTRUCT DECODER --------------------------------------------
# --------------------------------------------------------------

decInp = L.Input((MAX_SEQ_LEN, VOCAB_SIZE))
lstm1 = L.LSTM(VOCAB_SIZE, return_sequences=True)(decInp)
lstm2 = L.LSTM(VOCAB_SIZE, return_sequences=True)(lstm1)


# construct A branch
xa = L.Reshape((1, MAX_SEQ_LEN, VOCAB_SIZE))(lstm2)
xa = L.Conv2D(VOCAB_SIZE, (1, 1), padding='same', activation='relu')(xa)
currFeatMap0Dim = 1
currFeatMap1Dim = MAX_SEQ_LEN
kernelSize = [VOCAB_SIZE, VOCAB_SIZE] # [curr, next]
for iteration in range(maxIterations):
    if (kernelSize[0] > 8):
        kernelSize[1] = kernelSize[0] / 2
    if (currFeatMap0Dim < IMAGE_DIM): # perform a reduction in the 0 axis
        xa = halfExpansionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 0, f'dec_a{iteration}')(xa)
        currFeatMap0Dim *= 2
        kernelSize[0] = kernelSize[1] # perform update
    if (currFeatMap1Dim > IMAGE_DIM): # perform an expansion in the 1 axis
        xa = halfReductionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 1, f'dec_a{iteration}')(xa)
        currFeatMap1Dim /= 2
        kernelSize[0] = kernelSize[1] # perform update

# construct B branch
xb = L.Reshape((MAX_SEQ_LEN, 1, VOCAB_SIZE))(lstm2)
xb = L.Conv2D(VOCAB_SIZE, (1, 1), padding='same', activation='relu')(xb)
currFeatMap0Dim = MAX_SEQ_LEN
currFeatMap1Dim = 1
kernelSize = [VOCAB_SIZE, VOCAB_SIZE] # [curr, next]
for iteration in range(maxIterations):
    if (kernelSize[0] > 8):
        kernelSize[1] = kernelSize[0] / 2
    if (currFeatMap0Dim > IMAGE_DIM): # perform an expansion in the 1 axis
        xb = halfReductionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 0, f'dec_b{iteration}')(xb)
        currFeatMap0Dim /= 2
        kernelSize[0] = kernelSize[1] # perform update
    if (currFeatMap1Dim < IMAGE_DIM): # perform a reduction in the 0 axis
        xb = halfExpansionCell((currFeatMap0Dim, currFeatMap1Dim, kernelSize[0]), kernelSize[1], 1, f'dec_b{iteration}')(xb)
        currFeatMap1Dim *= 2
        kernelSize[0] = kernelSize[1] # perform update

join = L.Add()([xa, xb])
static1 = staticCell((IMAGE_DIM, IMAGE_DIM, 8), 8, 'dec_sta1')(join)
static2 = staticCell((IMAGE_DIM, IMAGE_DIM, 8), 3, 'dec_sta2')(static1)
decOut = L.Conv2D(NUM_CHANNELS, (1, 1), padding='same', activation='sigmoid', name='out')(static2)

decoder = Model(inputs=decInp, outputs=decOut, name='decoder')

# --------------------------------------------------------------
# CONSTRUCT AUTOENCODER ----------------------------------------
# --------------------------------------------------------------

inp = L.Input((IMAGE_DIM, IMAGE_DIM, NUM_CHANNELS))
encoded = encoder(inp)
out = decoder(encoded)
ae = keras.models.Model(inputs=inp, outputs=out)

ae.compile(optimizer='adam',
       loss='binary_crossentropy')

## Model Instantiation and Training

In [ ]:
history = ae.fit(X_train, 
                 X_train, 
                 validation_data=(X_valid, X_valid), epochs=70)

In [ ]:
plt.figure(figsize=(15, 5), dpi=400)
plt.plot(history.history['loss'], color='red', label='Train Loss')
plt.plot(history.history['val_loss'], color='blue', label='Valid Loss')
plt.grid()
plt.legend()
plt.show()

## Visualization

### Training Samples

In [ ]:
NUM_TRAIN_SAMPLES = 50
for i in range(NUM_TRAIN_SAMPLES):
    encoded = encoder.predict(X_train[i:i+1])
    pred = ae.predict(X_train[i:i+1])
    
    plt.figure(figsize=(20, 5), dpi=400)
    plt.subplot(1, 3, 1)
    plt.imshow(X_train[i], cmap='gray')
    plt.axis('off')
    plt.title('Ground Truth')
    plt.subplot(1, 3, 2)
    plt.imshow(pred[0], cmap='gray')
    plt.axis('off')
    plt.title('Reconstruction')
    plt.subplot(1, 3, 3)
    plt.imshow(encoded[0], cmap='magma')
    plt.title('Bottleneck Communication')
    plt.ylabel('Independent Sequences')
    plt.xlabel('Sequential Information')
    plt.show()

### Validation Samples

In [ ]:
NUM_VALID_SAMPLES = 50
for i in range(NUM_VALID_SAMPLES):
    encoded = encoder.predict(X_valid[i:i+1])
    pred = ae.predict(X_valid[i:i+1])
    
    plt.figure(figsize=(20, 5), dpi=400)
    plt.subplot(1, 3, 1)
    plt.imshow(X_valid[i], cmap='gray')
    plt.axis('off')
    plt.title('Ground Truth')
    plt.subplot(1, 3, 2)
    plt.imshow(pred[0], cmap='gray')
    plt.axis('off')
    plt.title('Reconstruction')
    plt.subplot(1, 3, 3)
    plt.imshow(encoded[0], cmap='magma')
    plt.title('Bottleneck Communication')
    plt.ylabel('Independent Sequences')
    plt.xlabel('Sequential Information')
    plt.show()

## Architecture Plotting

In [19]:
keras.utils.plot_model(encoder, show_shapes=True, dpi=400)

In [20]:
keras.utils.plot_model(decoder, show_shapes=True, dpi=400)

In [18]:
keras.utils.plot_model(ae, show_shapes=True, dpi=400)

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

In [17]:
ae.summary()